In [1]:
import scipy.io
import scipy.ndimage
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy import sparse
from scipy.linalg import block_diag
import pickle
import matplotlib as mpl
mpl.use('Agg')


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from timeit import default_timer
import torch
from functools import partial




from Solver import *


import sys
sys.path.append('../Utility')
import NeuralNet
import KalmanInversion 
from Numerics import interpolate_f2c, gradient_first_f2c
# import imp
# imp.reload(KalmanInversion )
# imp.reload(NeuralNet )



# jupyter nbconvert --to script 'Indirect_NN.ipynb'

# Reference quadratic function
 $$D(\theta) = \sqrt{\theta^2 + (\frac{\partial \theta}{\partial x})^2}$$

In [2]:
def permeability_ref(q, dq):
    return np.sqrt(q**2 + dq**2) 
def D_permeability_ref(q, dq):
    return q/np.sqrt(q**2 + dq**2), dq/np.sqrt(q**2 + dq**2)


# Generate Training data

In [3]:
def generate_data_helper(permeability, D_permeability, f_func, L=1.0, Nx = 100):
    xx = np.linspace(0.0, L, Nx)
    dy = xx[1] - xx[0]
    f = f_func(xx)   
    dbc = np.array([0.0, 0.0]) 
       
    model = lambda q, yy, res : nummodel(permeability_ref, q, yy, res)
    xx, t_data, q_data = explicit_solve(model, f, dbc, dt = 5.0e-6, Nt = 500000, save_every = 100000, L = L)

    
    print("Last step increment is : ", np.linalg.norm(q_data[-1, :] - q_data[-2, :]), " last step is : ", np.linalg.norm(q_data[-1, :]))
    
    q = q_data[-1, :]
    q_c, dq_c = interpolate_f2c(q), gradient_first_f2c(q, dy)
    return xx, f, q, q_c, dq_c 


f_funcs = []
n_data = 10

for i in range(1,n_data):
    def func(xx, A = i):
        return A * xx
    f_funcs.append(func)
    
    
L = 1.0
Nx = 100
n_data = len(f_funcs)
xx, f, q, q_c, dq_c = np.zeros((n_data, Nx)), np.zeros((n_data, Nx)), np.zeros((n_data, Nx)), np.zeros((n_data, Nx-1)), np.zeros((n_data, Nx-1))


for i in range(n_data):
    xx[i, :], f[i, :], q[i, :], q_c[i, :], dq_c[i, :] = generate_data_helper(permeability_ref, D_permeability_ref, f_funcs[i], L=L, Nx=Nx)
    

# visualize data

plt.figure()
for i in range(n_data):
    plt.plot(q_c[i, :], dq_c[i, :],  "--o", fillstyle="none")

plt.xlabel("q")
plt.ylabel("dq")
plt.title("Training Data")
plt.savefig("Poisson-Training-Data.png")

100000 max q 0.15420940171016215
200000 max q 0.16105500139707368
300000 max q 0.1612135690621972
400000 max q 0.16121705302977674
500000 max q 0.1612171295332711
Last step increment is :  5.244638612664234e-07  last step is :  1.0869268110500834
100000 max q 0.22587322936901522
200000 max q 0.22798575440830837
300000 max q 0.22799540970998744
400000 max q 0.22799545330412824
500000 max q 0.22799545350094425
Last step increment is :  1.349271491532571e-09  last step is :  1.537146653703915
100000 max q 0.2784558907343084
200000 max q 0.27923521393101597
300000 max q 0.27923626097074616
400000 max q 0.27923626237508703
500000 max q 0.27923626237669746
Last step increment is :  1.10940588520717e-11  last step is :  1.8826124807061453
100000 max q 0.32211004034818674
200000 max q 0.3224341061004881
300000 max q 0.3224342624262979
400000 max q 0.3224342625014399
500000 max q 0.3224342625014399
Last step increment is :  0.0  last step is :  2.173853645031209
100000 max q 0.3603452553980225


# Training Loss : || d(D dq/dy)/dy + f(x)|| on the quadratic function

In [4]:
def loss_aug(s_param, params):
    
    
    ind, outd, width   = s_param.ind, s_param.outd, s_param.width
    activation, initializer, outputlayer = s_param.activation, s_param.initializer, s_param.outputlayer
    
    dt, Nt, save_every = s_param.dt,  s_param.Nt,   s_param.save_every
    xx, f = s_param.xx, s_param.f
    dbc = s_param.dbc
    
    
    N_data, Nx = f.shape
    q_sol = np.zeros((N_data, Nx))
    
    net =  create_net(ind, outd, layers, width, activation, initializer, outputlayer,  params)
    nn_model = partial(nn_permeability, net=net, non_negative=True)
    model = lambda q, xx, res : nummodel(nn_model, q, xx, res)
    
    for i in range(N_data):
        _, t_data, q_data = explicit_solve(model, f[i, :], dbc, dt = dt, Nt = Nt, save_every = save_every, L = L)
        q_sol[i, :] = q_data[-1, :]
        
    return np.hstack((np.reshape(q_sol, -1), params))



## Start UKI

In [5]:
class PoissonParam:
    def __init__(self, 
                 xx, f, dbc, dt, Nt, save_every,
                 N_y, ind, outd, layers, width, activation, initializer, outputlayer 
                 ):
        self.theta_names = ["hyperparameters"]
        
        self.ind  = ind
        self.outd = outd
        self.width = width
        self.activation = activation
        self.initializer = initializer
        self.outputlayer = outputlayer
        
        self.dt = dt
        self.Nt = Nt
        self.save_every = save_every

        self.xx = xx
        self.f  = f
        self.dbc = dbc
        
        
        N_theta = ind*width + (layers - 2)*width**2 + width*outd + (layers - 1)*width + outd
        self.N_theta = N_theta
        
        
        self.N_y = N_y + N_theta

In [7]:
y = np.reshape(q, -1)
Sigma_eta = np.fabs(q)
for i in range(n_data):
    Sigma_eta[i, :] = np.mean(Sigma_eta[i, :])
Sigma_eta = np.diag(np.reshape((Sigma_eta*0.01)**2, -1))

dbc = np.array([0.0, 0.0])
N_y = len(y)
ind, outd, width = 2, 1, 10
layers = 2
activation, initializer, outputlayer = "sigmoid", "default", "None"
dt, Nt, save_every = 2.0e-6,  500000, 500000
s_param = PoissonParam(xx, f, dbc, dt, Nt, save_every,
                       N_y, ind, outd, layers, width, activation, initializer, outputlayer)


N_theta = s_param.N_theta

theta0_mean_init = NeuralNet.FNN(ind, outd, layers, width, activation, initializer, outputlayer).get_params()
theta0_mean = np.zeros(N_theta)

theta0_cov = np.zeros((N_theta, N_theta))
np.fill_diagonal(theta0_cov, 10.0**2)  
theta0_cov_init = np.zeros((N_theta, N_theta))
np.fill_diagonal(theta0_cov_init, 1.0**2)  

y_aug = np.hstack((y, theta0_mean))
Sigma_eta_aug = block_diag(Sigma_eta, theta0_cov)



alpha_reg = 1.0
update_freq = 1
N_iter = 100
gamma = 1.0

save_folder = "indirect_NN"
# uki_obj = KalmanInversion.UKI_Run(s_param, loss_aug, 
#     theta0_mean, theta0_mean_init, 
#     theta0_cov, theta0_cov_init, 
#     y_aug, Sigma_eta_aug,
#     alpha_reg,
#     gamma,
#     update_freq, 
#     N_iter,
#     save_folder = save_folder)


In [8]:
uki_obj = pickle.load( open( save_folder + "/ukiobj-" + str(16) + ".dat", "rb" ) )
trained_net = create_net(ind, outd, layers, width, activation, initializer, outputlayer, uki_obj.theta_mean[-1])

print(trained_net.modus['LinM{}'.format(1)].weight)
print(trained_net.modus['LinM{}'.format(1)].bias)
print(trained_net.modus['LinMout'].weight)
print(trained_net.modus['LinMout'].bias)

Parameter containing:
tensor([[15.0801, -0.1698],
        [15.8788, 10.1274],
        [ 0.9580, 14.1349],
        [ 3.9171,  0.1886],
        [ 3.8298, -3.5143],
        [ 3.6199, -7.8789],
        [-7.7524, -0.3156],
        [ 3.5713, -4.9080],
        [ 3.7240,  2.9567],
        [-2.1892, -1.4219]], requires_grad=True)
Parameter containing:
tensor([-1.1771, -2.5888, -9.7535, -3.5915,  1.8529,  4.9318, 12.4777, -0.4926,
        -7.5218, -2.5385], requires_grad=True)
Parameter containing:
tensor([[ 0.0766,  0.3347,  0.3761,  9.0216, -3.5489,  1.4512, -0.2568,  1.2489,
         -9.4000,  4.5770]], requires_grad=True)
Parameter containing:
tensor([0.7858], requires_grad=True)


# Direct test

In [9]:
N_test_1d = 200
L_test_1d = 1.0
N_test = N_test_1d**2

x1_test_1d = np.linspace(0, L_test_1d, N_test_1d)
x2_test_1d = np.linspace(-L_test_1d, L_test_1d, N_test_1d)
X_test_2d, Y_test_2d = np.meshgrid(x1_test_1d, x2_test_1d)

x_test = np.vstack((X_test_2d.reshape(-1), Y_test_2d.reshape(-1))).T    
y_test = permeability_ref(x_test[:, 0], x_test[:, 1]).reshape((N_test, 1))


y_pred = net_eval(trained_net, x_test)  

y_test_2d = y_test.reshape((N_test_1d,N_test_1d))
y_pred_2d = y_pred.reshape((N_test_1d,N_test_1d))


fig, ax = plt.subplots(ncols=3, nrows=1, figsize=((16,5)))


vmin, vmax = y_test_2d.min(), y_test_2d.max()
im0=ax[0].pcolormesh(X_test_2d, Y_test_2d, y_test_2d, vmin=vmin, vmax=vmax)
fig.colorbar(im0, ax=ax[0])
ax[0].set_title("Reference")

im1=ax[1].pcolormesh(X_test_2d, Y_test_2d, y_pred_2d, vmin=vmin, vmax=vmax)
fig.colorbar(im1, ax=ax[1])
ax[0].set_title("Prediction")

im2=ax[2].pcolormesh(X_test_2d, Y_test_2d, y_test_2d - y_pred_2d)
fig.colorbar(im2, ax=ax[2])
ax[0].set_title("Error")
fig.savefig("Poisson-Direct-Test.png")



/tmp/ipykernel_12585/479680617.py:23: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im0=ax[0].pcolormesh(X_test_2d, Y_test_2d, y_test_2d, vmin=vmin, vmax=vmax)
/tmp/ipykernel_12585/479680617.py:27: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  im1=ax[1].pcolormesh(X_test_2d, Y_test_2d, y_pred_2d, vmin=vmin, vmax=vmax)
/tmp/ipykernel_12585/479680617.py:31: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated si

# Plug-in test

In [19]:
def permeability_nn(q, dq):
    
    x = np.vstack((q, dq)).T
    
    permeability = nn_model(x, trained_net)  
    
    return permeability


L = 1.0
Nx = 100
xx_test = np.linspace(0.0, L, Nx)
f_test = 6*(1-2*xx_test)**2 - 2*(xx_test - xx_test**2)*(1 - 2*xx_test)**2 + 2*(xx_test - xx_test**2)**2 + 2 


f_test /= 2.0
dbc = np.array([0.0, 0.0]) 

MODEL = "exp_nummodel"

if MODEL == "exp_nummodel":
    
    model = lambda q, xx, res : nummodel(permeability_ref, q, xx, res)
    _, t_data_ref, q_data_ref = explicit_solve(model, f_test, dbc, dt = 5.0e-6, Nt = 200000, save_every = 100000, L = L)
    
    nn_model = partial(nn_permeability, net=trained_net, non_negative=False)
    model = lambda q, xx, res : nummodel(nn_model, q, xx, res)
    _, t_data, q_data = explicit_solve(model, f_test, dbc, dt = 5.0e-6, Nt = 200000, save_every = 100000, L = L)

else:
    print("ERROR")


plt.figure()
plt.plot(xx_test, q_data[-1, :],  "--o", fillstyle="none", label="Prediction")
plt.plot(xx_test, q_data_ref[-1, :],  "--*", label="Reference")

plt.xlabel("y")
plt.legend()

plt.savefig("Poisson-Indirect-Test.png")

100000 max q 0.27057425383730593
200000 max q 0.2714425871030778
100000 max q 0.264766007857636
200000 max q 0.26567915016675886
